# データの準備

In [36]:
%cd "/content/drive/MyDrive/Colab Notebooks/introduction_to_huggingface"

/content/drive/MyDrive/Colab Notebooks/introduction_to_huggingface


## ダウンロード・解凍

In [3]:
!wget -P data/ https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

--2022-03-01 07:40:25--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.228.200
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.228.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94688992 (90M) [application/x-gzip]
Saving to: ‘data/amazon_reviews_multilingual_JP_v1_00.tsv.gz’

amazon_reviews_mult 100%[===================>]  90.30M  66.9MB/s    in 1.3s    

2022-03-01 07:40:27 (66.9 MB/s) - ‘data/amazon_reviews_multilingual_JP_v1_00.tsv.gz’ saved [94688992/94688992]



In [4]:
!gunzip data/amazon_reviews_multilingual_JP_v1_00.tsv.gz

## 前処理

### 読み込み

In [5]:
import string
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
data_path = "data/amazon_reviews_multilingual_JP_v1_00.tsv"
df = pd.read_csv(data_path, sep="\t")

In [7]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,JP,65317,R33RSUD4ZTRKT7,B000001GBJ,957145596,SONGS FROM A SECRET GARDE,Music,1,1,15,N,Y,残念ながら…,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…,2012-12-05
1,JP,65317,R2U1VB8GPZBBEH,B000YPWBQ2,904244932,鏡の中の鏡‾ペルト作品集(SACD)(Arvo Part:Spiegel im Spiegel),Music,1,4,20,N,Y,残念ながら…,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…,2012-12-05
2,JP,65696,R1IBRCJPPGWVJW,B0002E5O9G,108978277,Les Miserables 10th Anniversary Concert,Music,5,2,3,N,Y,ドリームキャスト,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。,2013-03-02
3,JP,67162,RL02CW5XLYONU,B00004SRJ5,606528497,It Takes a Nation of Millions to Hold Us Back,Music,5,6,9,N,Y,やっぱりマスト,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...,2013-08-11
4,JP,67701,R2LA2SS3HU3A3L,B0093H8H8I,509738390,Intel CPU Core I3-3225 3.3GHz 3MBキャッシュ LGA1155...,PC,4,2,4,N,Y,コスパ的には十分,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。<br />...,2013-02-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262251,JP,49068176,R2M96OZ1BUXN0Q,B007VGXS8M,428248709,タイムトラベラーウォーリーをおえ！™ (Kindle Tablet Edition),Mobile_Apps,1,5,6,N,Y,起動できなくなる,とても面白いゲームなのに、一度でも終了すると二度と起動出来なくなる。<br />再インストー...,2013-07-14
262252,JP,49068176,R2OEBHIJOW83UZ,B00AB7IA1S,132300013,Kings Can Fly,Mobile_Apps,4,1,1,N,Y,パズル好きにはいい,ゆっくり時間を掛けてあれこれ進路を考えながら配置するのは結構楽しい作業。<br />なかなか...,2013-07-14
262253,JP,51365125,RV87LQ32MY9HF,B008B1BL4E,237522100,The Sleepwalkers: How Europe Went to War in 1914,Digital_Ebook_Purchase,5,2,2,N,N,Highly recommend,A very candid analisys as to how the WWI start...,2015-02-13
262254,JP,52617891,R3KFJLGX7XF2X4,B00R0K7VF0,835526164,The Buried Giant (English Edition),Digital_Ebook_Purchase,5,15,17,N,N,Magical,Could hardly put this novel down. Was sorry it...,2015-03-06


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262256 entries, 0 to 262255
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        262256 non-null  object
 1   customer_id        262256 non-null  int64 
 2   review_id          262256 non-null  object
 3   product_id         262256 non-null  object
 4   product_parent     262256 non-null  int64 
 5   product_title      262256 non-null  object
 6   product_category   262256 non-null  object
 7   star_rating        262256 non-null  int64 
 8   helpful_votes      262256 non-null  int64 
 9   total_votes        262256 non-null  int64 
 10  vine               262256 non-null  object
 11  verified_purchase  262256 non-null  object
 12  review_headline    262255 non-null  object
 13  review_body        262256 non-null  object
 14  review_date        262249 non-null  object
dtypes: int64(5), object(10)
memory usage: 30.0+ MB


In [11]:
review_and_rating_df = df[["review_body", "star_rating"]]

In [12]:
review_and_rating_df

,review_body,star_rating
0,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…,1
1,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…,1
2,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。,5
3,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...,5
4,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。<br />...,4
...,...,...
262251,とても面白いゲームなのに、一度でも終了すると二度と起動出来なくなる。<br />再インストー...,1
262252,ゆっくり時間を掛けてあれこれ進路を考えながら配置するのは結構楽しい作業。<br />なかなか...,4
262253,A very candid analisys as to how the WWI start...,5
262254,Could hardly put this novel down. Was sorry it...,5


### 英語の除去

In [15]:
def filter_by_ascii_rate(text, threshold=0.9):
    ascii_letters = set(string.printable)
    rate = sum(c in ascii_letters for c in text) / len(text)
    return rate <= threshold

In [18]:
is_jp = review_and_rating_df["review_body"].apply(filter_by_ascii_rate)
review_and_rating_df = review_and_rating_df[is_jp].reset_index(drop=True)

In [19]:
review_and_rating_df

,review_body,star_rating
0,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…,1
1,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…,1
2,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。,5
3,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...,5
4,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。<br />...,4
...,...,...
256445,通称「メント」。ポストメタリカの3Mの一角を担うバンド。#1.Practice What Y...,5
256446,久しぶりにフレディを聞きたくなり購入。さまざまなご意見はありますが、ロックとオペラの融合、そ...,5
256447,とても面白いゲームなのに、一度でも終了すると二度と起動出来なくなる。<br />再インストー...,1
256448,ゆっくり時間を掛けてあれこれ進路を考えながら配置するのは結構楽しい作業。<br />なかなか...,4


### HTMLタグの除去

In [20]:
def clean_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(strip=True)
    return text

In [23]:
review_and_rating_df["review_body"] = review_and_rating_df["review_body"].apply(clean_html)

In [24]:
review_and_rating_df

,review_body,star_rating
0,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…,1
1,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…,1
2,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。,5
3,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...,5
4,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。動画のエンコ...,4
...,...,...
256445,通称「メント」。ポストメタリカの3Mの一角を担うバンド。#1.Practice What Y...,5
256446,久しぶりにフレディを聞きたくなり購入。さまざまなご意見はありますが、ロックとオペラの融合、そ...,5
256447,とても面白いゲームなのに、一度でも終了すると二度と起動出来なくなる。再インストールすれば一度...,1
256448,ゆっくり時間を掛けてあれこれ進路を考えながら配置するのは結構楽しい作業。なかなか難しいと思っ...,4


### ラベルの修正

In [25]:
review_and_rating_df["star_rating"] = review_and_rating_df["star_rating"] - 1

In [26]:
review_and_rating_df

,review_body,star_rating
0,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…,0
1,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…,0
2,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。,4
3,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...,4
4,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。動画のエンコ...,3
...,...,...
256445,通称「メント」。ポストメタリカの3Mの一角を担うバンド。#1.Practice What Y...,4
256446,久しぶりにフレディを聞きたくなり購入。さまざまなご意見はありますが、ロックとオペラの融合、そ...,4
256447,とても面白いゲームなのに、一度でも終了すると二度と起動出来なくなる。再インストールすれば一度...,0
256448,ゆっくり時間を掛けてあれこれ進路を考えながら配置するのは結構楽しい作業。なかなか難しいと思っ...,3


## 保存

In [35]:
review_and_rating_df.to_csv("data/amazon_reveiws_and_ratings.tsv", sep="\t", index=False)